In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.fftpack import fft, ifft, fftfreq
from scipy.io import wavfile
import matplotlib.pyplot as plt
from tqdm import tqdm
import vmdpy

In [ ]:
csv_file_path = r"./neurovoz_v3/data/audio_features/audio_features.csv"
audio_directory = r"./neurovoz_v3/data/audios"
imf_directory = r"./Outputs/VMD/IMFs"
residual_directory = r"./Outputs/VMD/Residual"
reconstructed_directory = r"./Outputs/VMD/Reconstructed_Signal"
plot_directory = r"./Outputs/VMD/Plots"
df = pd.read_csv(csv_file_path)

In [ ]:
def vmd(signal, alpha=120, tau=0, K=5, DC=0, init=1, tol=1e-7):
    print("VMD started")
    # Ensure signal is a numpy array
    signal = np.array(signal, dtype=np.float64)
    
    # Normalize the signal (optional)
    signal = signal / np.max(np.abs(signal))

    # Perform VMD
    modes, _, _ = vmdpy.VMD(signal, alpha, tau, K, DC, init, tol)
    print(modes.shape)
    # Reconstruct the signal from the modes
    reconstructed_signal = np.sum(modes, axis=0, dtype=np.float64)
    print(reconstructed_signal.shape)
    # Calculate residual (signal minus reconstructed signal)
    residual = signal - reconstructed_signal
    print("VMD completed")
    return modes, residual


In [ ]:
import torch

def VMD_torch(f, alpha, tau, K, DC, init, tol, device="cuda"):
    """
    Variational Mode Decomposition (VMD) implemented in PyTorch with device-agnostic initialization
    and optimized FFT operations.

    Parameters:
    ---------
    f : numpy.ndarray
        Input signal (1D array).
    alpha : float
        Bandwidth constraint parameter.
    tau : float
        Lagrange multiplier for enforcing signal reconstruction fidelity.
    K : int
        Number of modes to decompose the signal into.
    DC : bool
        If True, constrain the first mode to have zero mean (DC component).
    init : int
        Initialization mode for center frequencies:
        - 1: Linear initialization.
        - 2: Random logarithmic initialization.
        - 0: All frequencies start at zero.
    tol : float
        Convergence tolerance.
    device : str
        Target device for computation ('cuda' or 'cpu').

    Returns:
    --------
    u : numpy.ndarray
        Decomposed modes in the time domain.
    u_hat_final : numpy.ndarray
        Fourier-domain representation of decomposed modes.
    omega : numpy.ndarray
        Center frequencies of the modes.
    """
    device = torch.device("cuda" if device == "cuda" and torch.cuda.is_available() else "cpu")
    f = torch.tensor(f, device=device, dtype=torch.float32).contiguous()
    
    # Ensure even length for the input signal
    if f.numel() % 2 != 0:
        print("Input signal must have an even length.")
        f = f[:-1]

    # Period and sampling frequency
    fs = 1.0 / len(f)
    ltemp = len(f) // 2

    # Mirrored signal
    fMirr = torch.cat([f[:ltemp].flip(0), f, f[-ltemp:].flip(0)], dim=0)

    # Time and spectral domain discretization
    T = len(fMirr)
    freqs = torch.arange(1, T + 1, device=device) / T - 0.5 - (1 / T)

    # FFT of the mirrored signal
    f_hat = torch.fft.fftshift(torch.fft.fft(fMirr))
    f_hat_plus = f_hat.clone()
    f_hat_plus[:T // 2] = 0

    # Initialize omega
    if init == 1:
        omega_curr = torch.linspace(0, 0.5, K, device=device)
    elif init == 2:
        omega_curr = torch.sort(
            torch.exp(torch.log(fs) + (torch.log(0.5) - torch.log(fs)) * torch.rand(K, device=device))
        )[0]
    else:
        omega_curr = torch.zeros(K, device=device)
    if DC:
        omega_curr[0] = 0

    # Initialize variables
    lambda_curr = torch.zeros(T, dtype=torch.cfloat, device=device)
    u_curr = torch.zeros((T, K), dtype=torch.cfloat, device=device)
    u_prev = torch.zeros((T, K), dtype=torch.cfloat, device=device)
    omega_history = torch.zeros((500, K), device=device)

    # Convergence parameters
    uDiff = tol + torch.finfo(torch.float32).eps
    n = 0
    Niter = 500

    while uDiff > tol and n < Niter - 1:
        u_prev.copy_(u_curr)
        sum_uk = torch.sum(u_prev, dim=1, keepdim=True) - u_prev[:, 0:1]

        # Update first mode
        u_curr[:, 0] = (f_hat_plus - sum_uk.squeeze() - lambda_curr / 2) / (1 + alpha * (freqs - omega_curr[0]) ** 2)
        if not DC:
            omega_curr[0] = torch.sum(
                freqs[T // 2:] * torch.abs(u_curr[T // 2:, 0]) ** 2
            ) / torch.sum(torch.abs(u_curr[T // 2:, 0]) ** 2)

        # Update remaining modes
        for k in range(1, K):
            sum_uk += u_curr[:, k - 1:k] - u_prev[:, k:k + 1]
            u_curr[:, k] = (f_hat_plus - sum_uk.squeeze() - lambda_curr / 2) / (
                1 + alpha * (freqs - omega_curr[k]) ** 2
            )
            omega_curr[k] = torch.sum(
                freqs[T // 2:] * torch.abs(u_curr[T // 2:, k]) ** 2
            ) / torch.sum(torch.abs(u_curr[T // 2:, k]) ** 2)

        # Update Lagrange multiplier
        lambda_curr += tau * (torch.sum(u_curr, dim=1) - f_hat_plus)

        # Check convergence
        omega_history[n + 1] = omega_curr
        n += 1
        uDiff = torch.sqrt(torch.sum(torch.abs(u_curr - u_prev) ** 2) / u_curr.numel())

    # Truncate omega history
    omega = omega_history[:n]

    # Reconstruct signal modes in the time domain
    u_hat = torch.zeros((T, K), dtype=torch.cfloat, device=device)
    u_hat[T // 2:, :] = u_curr[T // 2:, :]
    u_hat[:T // 2, :] = torch.conj(u_curr[T // 2:T, :].flip(0))  # Fix slicing range
    u = torch.real(torch.fft.ifft(torch.fft.ifftshift(u_hat, dim=0), dim=0))

        # Extract the relevant part of the signal with robust slicing
    start = max(0, T // 4)
    end = min(u.shape[1], 3 * T // 4)
    if start >= end:
        raise ValueError(f"Invalid slicing range: start={start}, end={end}, T={T}")
    u = u[:, start:end]

    # Assert non-empty result
    assert u.numel() > 0, "u tensor is empty after slicing! Check input signal and slicing range."


    # Fourier-domain representation of the final modes
    u_hat_final = torch.fft.fftshift(torch.fft.fft(u, dim=0), dim=0)


    return u.cpu().numpy(), u_hat_final.cpu().numpy(), omega.cpu().numpy()




# Helper function to check GPU memory
def check_gpu_memory():
    """
    Print current GPU memory usage
    """
    mem_info = cp.cuda.runtime.memGetInfo()
    free_mem = mem_info[0] / 1024**2  # Convert to MB
    total_mem = mem_info[1] / 1024**2
    used_mem = total_mem - free_mem
    print(f"GPU Memory Usage:")
    print(f"Total: {total_mem:.2f} MB")
    print(f"Used:  {used_mem:.2f} MB")
    print(f"Free:  {free_mem:.2f} MB")

# Example usage
if __name__ == "__main__":
    # Generate sample signal
    t = np.linspace(0, 1, 1000)
    f1, f2 = 2, 10
    signal = np.sin(2*np.pi*f1*t) + np.sin(2*np.pi*f2*t)
    
    # VMD parameters
    alpha = 2000
    tau = 0
    K = 2
    DC = False
    init = 1
    
    # Decompose signal
    modes, spectra, frequencies = VMD_torch(signal, alpha, tau, K, DC, init, 1e-6)
    
    print(f"Number of modes extracted: {modes.shape[0]}")
    print(f"Length of each mode: {modes.shape[1]}")
    print(f"Final center frequencies: {frequencies[-1]}")

In [ ]:
def save_modes_and_reconstruct(modes, residual, sample_rate, original_signal, base_filename):
    """Save modes, residual, and reconstructed signal with length adjustment."""
    
    reconstructed_signal = np.zeros_like(original_signal, dtype=np.float64)
    
    for i, mode in enumerate(modes[:5]):  
    
        mode = np.pad(mode, (0, len(original_signal) - len(mode)), 'constant')[:len(original_signal)]
        mode_path = os.path.join(imf_directory, f"{base_filename}_mode_{i+1}.wav")
        mode = (mode / np.max(np.abs(mode))) * 32767 if np.max(np.abs(mode)) > 0 else mode  # Normalize mode
        mode = mode.astype(np.int16)
        wavfile.write(mode_path, sample_rate, mode)
        reconstructed_signal += mode

    residual = np.pad(residual, (0, len(original_signal) - len(residual)), 'constant')[:len(original_signal)]
    residual_path = os.path.join(residual_directory, f"{base_filename}_residual.wav")
    wavfile.write(residual_path, sample_rate, residual.astype(np.int16))

    reconstructed_signal += residual
    if np.max(np.abs(reconstructed_signal)) > 0:
        reconstructed_signal /= np.max(np.abs(reconstructed_signal)) 

    reconstructed_path = os.path.join(reconstructed_directory, f"{base_filename}_reconstructed.wav")
    wavfile.write(reconstructed_path, sample_rate, (reconstructed_signal * 32767).astype(np.int16))

    plt.figure(figsize=(10, 10))
    for i, mode in enumerate(modes[:5]):
        plt.subplot(6, 1, i+1)
        plt.plot(mode)
        plt.title(f"Mode {i+1}")
    plt.subplot(6, 1, 6)
    plt.plot(residual)
    plt.title("Residual")
    plt.tight_layout()
    plot_path = os.path.join(plot_directory, f"{base_filename}_modes_plot.png")
    plt.savefig(plot_path)
    plt.close()


In [ ]:
def save_modes_and_reconstruct_vmd(modes, sample_rate, original_signal, base_filename):

    # Initialize reconstructed signal
    reconstructed_signal = np.zeros_like(original_signal, dtype=np.float64)

    # Save each mode
    for i, mode in enumerate(modes):
        # Adjust mode length to match original signal
        mode = np.pad(mode, (0, len(original_signal) - len(mode)), 'constant')[:len(original_signal)]

        # Save mode as WAV
        mode_path = os.path.join(imf_directory, f"{base_filename}_mode_{i+1}.wav")
        if np.max(np.abs(mode)) > 0:
            mode = (mode / np.max(np.abs(mode))) * 32767  # Normalize mode
        mode = mode.astype(np.int16)
        wavfile.write(mode_path, sample_rate, mode)

        # Add to reconstructed signal
        reconstructed_signal += mode

    # # Save residual
    # residual = np.pad(residual, (0, len(original_signal) - len(residual)), 'constant')[:len(original_signal)]
    # residual_path = os.path.join(residual_directory, f"{base_filename}_residual.wav")
    # residual = (residual / np.max(np.abs(residual))) * 32767 if np.max(np.abs(residual)) > 0 else residual
    # wavfile.write(residual_path, sample_rate, residual.astype(np.int16))

    # Add residual to reconstructed signal
    reconstructed_signal += residual
    if np.max(np.abs(reconstructed_signal)) > 0:
        reconstructed_signal /= np.max(np.abs(reconstructed_signal))

    # Save reconstructed signal
    reconstructed_path = os.path.join(reconstructed_directory, f"{base_filename}_reconstructed.wav")
    wavfile.write(reconstructed_path, sample_rate, (reconstructed_signal * 32767).astype(np.int16))

    # Plot modes and residual
    plt.figure(figsize=(10, 10))
    for i, mode in enumerate(modes[:5]):  # Limit to the first 5 modes for plotting
        plt.subplot(6, 1, i+1)
        plt.plot(mode, label=f"Mode {i+1}")
        plt.legend(loc='upper right')
    plt.subplot(6, 1, 6)
    plt.plot(residual, label="Residual", color='red')
    plt.legend(loc='upper right')
    plt.tight_layout()

    # Save plot
    plot_path = os.path.join(plot_directory, f"{base_filename}_modes_plot.png")
    plt.savefig(plot_path)
    plt.close()


In [ ]:
def process_audio_file(row):
    """
    Process each audio file for VMD (Variational Mode Decomposition).
    
    Parameters:
        row (dict): A dictionary containing audio file metadata (e.g., path).
    """
    # Extract and clean the relative path
    relative_path = row['AudioPath'].strip()
    
    if relative_path.startswith('../data/audios/'):
        relative_path = relative_path.replace('../data/audios/', '')

    # Construct the full file path
    file_path = os.path.join(audio_directory, relative_path)
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    
    try:
        # Read the audio file
        sample_rate, data = wavfile.read(file_path)

        # Convert stereo to mono if needed
        if len(data.shape) == 2: 
            data = data[:, 0]
            

        # Normalize the data
        data = data / np.max(np.abs(data))

        # # Perform VMD
        modes, residual, _ = VMD_torch(
            f = data, 
            alpha=data.shape[0],  # adjust as needed
            tau=0,       # noise-slack
            K=8,         # number of modes
            DC=False,     # keep zero frequency mode
            init=1,      # uniformly distributed initial omegas
            tol=1e-30     # convergence tolerance
        )

        # # Save the decomposed modes and reconstructed signal
        # save_modes_and_reconstruct(modes, residual, sample_rate, data, base_filename)

        # print(f"Processed {base_filename}")

    except Exception as e:
        print(f"Error processing {file_path}: {e}")


In [ ]:
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    # Extract and clean the relative path
    relative_path = row['AudioPath'].strip()
    
    if relative_path.startswith('../data/audios/'):
        relative_path = relative_path.replace('../data/audios/', '')

    # Construct the full file path
    file_path = os.path.join(audio_directory, relative_path)
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    
    try:
        # Read the audio file
        sample_rate, data = wavfile.read(file_path)

        # Convert stereo to mono if needed
        if len(data.shape) == 2: 
            data = data[:, 0]
            

        # Normalize the data
        data = data / np.max(np.abs(data))

        # # Perform VMD
        modes, residual, _ = VMD_torch(
            f = data, 
            alpha=data.shape[0],  # adjust as needed
            tau=0,       # noise-slack
            K=8,         # number of modes
            DC=False,     # keep zero frequency mode
            init=1,      # uniformly distributed initial omegas
            tol=1e-30     # convergence tolerance
        )

        # # Save the decomposed modes and reconstructed signal
        # save_modes_and_reconstruct(modes, residual, sample_rate, data, base_filename)

        # print(f"Processed {base_filename}")

    except Exception as e:
        print(f"Error processing {file_path}: {e}")


In [ ]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy.io import wavfile

# Directory containing subfolders with audio files
audio_directory = r"C:\Users\Asus\OneDrive - Amrita Vishwa Vidyapeetham\Desktop\biorun\IMFs\IMFS_VMD"

# Directory to save Mel spectrograms
spectrogram_directory = r"C:\Users\Asus\OneDrive - Amrita Vishwa Vidyapeetham\Desktop\biorun\Spectrograms\VMD_S"

# Create the spectrogram directory if it doesn't exist
if not os.path.exists(spectrogram_directory):
    os.makedirs(spectrogram_directory)

# Function to compute and save Mel spectrogram
def save_mel_spectrogram(audio_data, sample_rate, output_path):
    # Compute the Mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate, n_mels=128, fmax=8000)

    # Convert power spec to dB for visualization
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Plot the Mel spectrogram
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spectrogram_db, sr=sample_rate, x_axis='time', y_axis='mel', fmax=8000)

    # Remove the title, axis labels, and color bar
    plt.axis('off')

    # Save the plot
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0, transparent=True)
    plt.close()

# Function to process each audio file in the subfolders
def process_audio_file(row):
    """
    Process each audio file for VMD and generate Mel spectrogram.
    
    Parameters:
        row (dict): A dictionary containing audio file metadata (e.g., path).
    """
    # Extract and clean the relative path
    relative_path = row['AudioPath'].strip()
    if relative_path.startswith('../data/audios/'):
        relative_path = relative_path.replace('../data/audios/', '')

    # Construct the full file path
    file_path = os.path.join(audio_directory, relative_path)
    base_filename = os.path.splitext(os.path.basename(file_path))[0]

    try:
        # Read the audio file
        sample_rate, data = wavfile.read(file_path)

        # Convert stereo to mono if needed
        if len(data.shape) == 2: 
            data = data[:, 0]

        # Normalize the data
        data = data / np.max(np.abs(data))

        # Create a folder to save the Mel spectrograms for this specific subfolder
        subfolder_name = os.path.basename(os.path.dirname(file_path))
        output_folder = os.path.join(spectrogram_directory, subfolder_name)

        # Create the subfolder if it doesn't exist
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Create the output file path for the Mel spectrogram
        spectrogram_file_name = base_filename + '_mel_spectrogram.png'
        spectrogram_file_path = os.path.join(output_folder, spectrogram_file_name)

        # Compute and save Mel spectrogram
        save_mel_spectrogram(data, sample_rate, spectrogram_file_path)

        print(f"Mel Spectrogram saved: {spectrogram_file_path}")

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Iterate through each subfolder and audio file
for root, dirs, files in os.walk(audio_directory):
    for file in files:
        if file.endswith('.wav'):
            # Construct the relative path for the file
            relative_path = os.path.relpath(os.path.join(root, file), audio_directory)
            row = {'AudioPath': relative_path}
            
            # Process the audio file
            process_audio_file(row)


In [ ]:
import torch
import torch.fft

def vmd(signal, alpha=2000, tau=0, K=5, DC=0, init=None, tol=1e-6, device='cuda'):
    """
    GPU-accelerated Variational Mode Decomposition (VMD)
    
    Args:
        signal: Input signal (1D tensor).
        alpha: Regularization parameter.
        tau: Time step for dual ascent.
        K: Number of modes.
        DC: 0 for no DC component, 1 otherwise.
        init: Initial values for modes and Lagrange multiplier.
        tol: Convergence tolerance.
        device: 'cuda' for GPU acceleration, 'cpu' otherwise.
    
    Returns:
        modes: Decomposed modes (K x N tensor).
    """
    # Move input signal to device
    signal = torch.tensor(signal, dtype=torch.float32, device=device)
    N = len(signal)
    
    # Fourier transform of the input signal
    f_signal = torch.fft.fft(signal)
    omega = torch.fft.fftfreq(N, d=1.0)
    omega = omega.to(device)
    
    # Initialization
    if init is None:
        u = torch.zeros((K, N), device=device, dtype=torch.complex64)
        omega_k = torch.linspace(0, torch.max(omega), K, device=device)
    else:
        u, omega_k = init
    
    # Lagrange multiplier
    lambda_hat = torch.zeros(N, device=device, dtype=torch.complex64)
    
    # Dual ascent iterations
    iteration = 0
    while True:
        u_prev = u.clone()
        
        # Update each mode
        for k in range(K):
            # Construct denominator term
            omega_shifted = omega - omega_k[k]
            denominator = 1 + alpha * (omega_shifted ** 2)
            
            # Compute mode in Fourier domain
            f_u_k = (f_signal - lambda_hat - torch.sum(u, dim=0) + u[k]) / denominator
            u[k] = torch.fft.ifft(f_u_k).real
        
        # Update omega_k
        for k in range(K):
            omega_k[k] = torch.sum(torch.abs(u[k]) ** 2 * omega) / torch.sum(torch.abs(u[k]) ** 2)
        
        # Update Lagrange multiplier
        residual = signal - torch.sum(u, dim=0).real
        lambda_hat += tau * torch.fft.fft(residual)
        
        # Check convergence
        if torch.norm(u - u_prev) < tol:
            break
        iteration += 1
        if iteration > 500:  # Prevent infinite loops
            break
    
    return u.real.cpu().detach().numpy()



# Example usage
if __name__ == "__main__":
    import numpy as np
    import matplotlib.pyplot as plt

    # Generate synthetic signal
    t = np.linspace(0, 1, 1000)
    signal = np.sin(2 * np.pi * 10 * t) + 0.5 * np.sin(2 * np.pi * 50 * t)
    
    # Run GPU-accelerated VMD
    modes = vmd(signal, K=5, device='cuda')

    # Plot results
    plt.figure(figsize=(25, 4))
    plt.plot(t, signal, label="Original Signal")
    for i, mode in enumerate(modes):
        plt.plot(t, mode, label=f"Mode {i+1}")
    plt.legend()
    plt.show()


In [ ]:
import numpy as np
import time
from vmdpy import VMD
import torch
import torch.fft

# VMDPy (CPU-based)
def vmdpy_vmd(signal, alpha=2000, tau=0, K=5, DC=0, tol=1e-6):
    """
    VMD using the vmdpy library (CPU implementation).
    """
    u, a, b = VMD(signal, alpha, tau, K, DC, init=1, tol=tol)
    return u, a, b

# GPU-Accelerated VMD
def gpu_vmd(signal, alpha=2000, tau=0, K=5, DC=0, tol=1e-6, device='cuda'):
    signal = torch.tensor(signal, dtype=torch.float32, device=device)
    N = len(signal)
    f_signal = torch.fft.fft(signal)
    omega = torch.fft.fftfreq(N, d=1.0).to(device)

    u = torch.zeros((K, N), device=device, dtype=torch.complex64)
    omega_k = torch.linspace(0, torch.max(omega), K, device=device)
    lambda_hat = torch.zeros(N, device=device, dtype=torch.complex64)

    iteration = 0
    while True:
        u_prev = u.clone()
        for k in range(K):
            omega_shifted = omega - omega_k[k]
            denominator = 1 + alpha * (omega_shifted ** 2)
            f_u_k = (f_signal - lambda_hat - torch.sum(u, dim=0) + u[k]) / denominator
            u[k] = torch.fft.ifft(f_u_k).real
        for k in range(K):
            omega_k[k] = torch.sum(torch.abs(u[k]) ** 2 * omega) / torch.sum(torch.abs(u[k]) ** 2)
        residual = signal - torch.sum(u, dim=0).real
        lambda_hat += tau * torch.fft.fft(residual)
        if torch.norm(u - u_prev) < tol or iteration > 500:
            break
        iteration += 1

    return u.real.cpu().detach().numpy()

def pad_signal(original, reconstructed):
    if len(reconstructed) < len(original):
        padding = len(original) - len(reconstructed)
        reconstructed = np.pad(reconstructed, (0, padding), mode='constant', constant_values=0)
    elif len(reconstructed) > len(original):
        reconstructed = reconstructed[:len(original)]
    return reconstructed

# Benchmarking
if __name__ == "__main__":
    import matplotlib.pyplot as plt
    from sklearn.metrics import mean_squared_error
    from scipy.io import wavfile
    from VMD.main import GPU_VMD
    
    signal = wavfile.read(r'C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0036.wav')[1]

    # Generate a synthetic signal
    t = np.linspace(0, 1, len(signal))
    # signal = np.sin(2 * np.pi * 10 * t) + 0.5 * np.sin(2 * np.pi * 50 * t)
    print("signal len", len(signal))
    # VMDPy
    start_time = time.time()
    modes_vmdpy, a ,b = vmdpy_vmd(signal, alpha=200000, tau=0, K=8, DC=False, tol=1e-70)
    vmdpy_time = time.time() - start_time

    # GPU-Accelerated VMD
    # Initialize VMD
    vmd = GPU_VMD(
        alpha=200000,  # Bandwidth constraint
        tau=0,       # Noise-tolerance
        n_modes=8,   # Number of modes to extract
        dc_component=False,  # Whether to force first mode to be DC
        init_method=1,      # Frequency initialization method
        tolerance=1e-6,     # Convergence tolerance
        device='cuda'       # Computing device
    )
    start_time = time.time()
    # Decompose signal
    result = vmd.decompose(signal)
    
    
    modes_gpu = result.modes

    

    gpu_time = time.time() - start_time

    # Calculate reconstruction error
    reconstructed_vmdpy = np.sum(modes_vmdpy, axis=0)
    reconstructed_gpu = np.sum(modes_gpu, axis=0)

    # Pad the reconstructed signals
    reconstructed_vmdpy_padded = pad_signal(signal, reconstructed_vmdpy)
    reconstructed_gpu_padded = pad_signal(signal, reconstructed_gpu)

    # Calculate mean squared errors
    mse_vmdpy = mean_squared_error(signal, reconstructed_vmdpy_padded)
    mse_gpu = mean_squared_error(signal, reconstructed_gpu_padded)

    # Print results
    print(f"VMDPy (CPU): Time = {vmdpy_time:.4f}s, MSE = {mse_vmdpy:.4e}")
    print(f"GPU-Accelerated VMD: Time = {gpu_time:.4f}s, MSE = {mse_gpu:.4e}")

    # Plot VMDPy results (separate modes)
    plt.figure(figsize=(25, 18))
    plt.suptitle("Decomposed Modes: VMDPy (CPU Implementation)", fontsize=16)
    for i, mode in enumerate(modes_vmdpy):
        plt.subplot(len(modes_vmdpy), 1, i + 1)
        plt.plot(t, pad_signal(signal, mode), label=f"VMDPy Mode {i+1}", color='blue')
        plt.legend()
        plt.xlabel("Time")
        plt.ylabel("Amplitude")
        plt.grid(True)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

    # Plot GPU-Accelerated VMD results (separate modes)
    plt.figure(figsize=(25, 18))
    plt.suptitle("Decomposed Modes: GPU-Accelerated VMD", fontsize=16)
    for i, mode in enumerate(modes_gpu):
        plt.subplot(len(modes_gpu), 1, i + 1)
        plt.plot(t, pad_signal(signal, mode), label=f"GPU Mode {i+1}", color='green')
        plt.legend()
        plt.xlabel("Time")
        plt.ylabel("Amplitude")
        plt.grid(True)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()




In [ ]:
import numpy as np
import time
from vmdpy import VMD
import torch
import torch.fft

# VMDPy (CPU-based)
def vmdpy_vmd(signal, alpha=2000, tau=0, K=5, DC=0, tol=1e-6):
    """
    VMD using the vmdpy library (CPU implementation).
    """
    u, a, b = VMD(signal, alpha, tau, K, DC, init=1, tol=tol)
    return u, a, b

# GPU-Accelerated VMD
def gpu_vmd(signal, alpha=2000, tau=0, K=5, DC=0, tol=1e-6, device='cuda'):
    signal = torch.tensor(signal, dtype=torch.float32, device=device)
    N = len(signal)
    f_signal = torch.fft.fft(signal)
    omega = torch.fft.fftfreq(N, d=1.0).to(device)

    u = torch.zeros((K, N), device=device, dtype=torch.complex64)
    omega_k = torch.linspace(0, torch.max(omega), K, device=device)
    lambda_hat = torch.zeros(N, device=device, dtype=torch.complex64)

    iteration = 0
    while True:
        u_prev = u.clone()
        for k in range(K):
            omega_shifted = omega - omega_k[k]
            denominator = 1 + alpha * (omega_shifted ** 2)
            f_u_k = (f_signal - lambda_hat - torch.sum(u, dim=0) + u[k]) / denominator
            u[k] = torch.fft.ifft(f_u_k).real
        for k in range(K):
            omega_k[k] = torch.sum(torch.abs(u[k]) ** 2 * omega) / torch.sum(torch.abs(u[k]) ** 2)
        residual = signal - torch.sum(u, dim=0).real
        lambda_hat += tau * torch.fft.fft(residual)
        if torch.norm(u - u_prev) < tol or iteration > 500:
            break
        iteration += 1

    return u.real.cpu().detach().numpy()

def pad_signal(original, reconstructed):
    if len(reconstructed) < len(original):
        padding = len(original) - len(reconstructed)
        reconstructed = np.pad(reconstructed, (0, padding), mode='constant', constant_values=0)
    elif len(reconstructed) > len(original):
        reconstructed = reconstructed[:len(original)]
    return reconstructed

# Benchmarking
if __name__ == "__main__":
    import matplotlib.pyplot as plt
    from sklearn.metrics import mean_squared_error
    from scipy.io import wavfile
    
    signal = wavfile.read(r'C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0034.wav')[1]

    # Generate a synthetic signal
    t = np.linspace(0, 1, len(signal))
    # signal = np.sin(2 * np.pi * 10 * t) + 0.5 * np.sin(2 * np.pi * 50 * t)
    print("signal len", len(signal))
    # VMDPy
    start_time = time.time()
    modes_vmdpy, a ,b = vmdpy_vmd(signal, alpha=200000, tau=0, K=7, DC=False, tol=1e-70)
    vmdpy_time = time.time() - start_time

    # GPU-Accelerated VMD
    start_time = time.time()
    modes_gpu, c,d = VMD_torch(
    f = signal, 
    alpha=200000,  # adjust as needed
    tau=0,       # noise-slack
    K=7,         # number of modes
    DC=False,     # keep zero frequency mode
    init=1,      # uniformly distributed initial omegas
    tol=1e-70     # convergence tolerance
)
    
    # modes_gpu = np.insert(modes_gpu, 3, modes_gpu[-1], axis=0)[:-1]
    #f, alpha, tau, K, DC, init, tol, device="cuda"
    gpu_time = time.time() - start_time

    # Calculate reconstruction error
    reconstructed_vmdpy = np.sum(modes_vmdpy, axis=0)
    reconstructed_gpu = np.sum(modes_gpu, axis=0)

    # Pad the reconstructed signals
    reconstructed_vmdpy_padded = pad_signal(signal, reconstructed_vmdpy)
    reconstructed_gpu_padded = pad_signal(signal, reconstructed_gpu)

    # Calculate mean squared errors
    mse_vmdpy = mean_squared_error(signal, reconstructed_vmdpy_padded)
    mse_gpu = mean_squared_error(signal, reconstructed_gpu_padded)

    # Print results
    print(f"VMDPy (CPU): Time = {vmdpy_time:.4f}s, MSE = {mse_vmdpy:.4e}")
    print(f"GPU-Accelerated VMD: Time = {gpu_time:.4f}s, MSE = {mse_gpu:.4e}")

    # Plot VMDPy results (separate modes)
    plt.figure(figsize=(25, 18))
    plt.suptitle("Decomposed Modes: VMDPy (CPU Implementation)", fontsize=16)
    for i, mode in enumerate(modes_vmdpy):
        plt.subplot(len(modes_vmdpy), 1, i + 1)
        plt.plot(t, pad_signal(signal, mode), label=f"VMDPy Mode {i+1}", color='blue')
        plt.legend()
        plt.xlabel("Time")
        plt.ylabel("Amplitude")
        plt.grid(True)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

    # Plot GPU-Accelerated VMD results (separate modes)
    plt.figure(figsize=(25, 18))
    plt.suptitle("Decomposed Modes: GPU-Accelerated VMD", fontsize=16)
    for i, mode in enumerate(modes_gpu):
        plt.subplot(len(modes_gpu), 1, i + 1)
        plt.plot(t, pad_signal(signal, mode), label=f"GPU Mode {i+1}", color='green')
        plt.legend()
        plt.xlabel("Time")
        plt.ylabel("Amplitude")
        plt.grid(True)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()




In [ ]:
# Plot VMDPy results (separate modes)
plt.figure(figsize=(25, 18))
plt.suptitle("Decomposed Modes: VMDPy (CPU Implementation)", fontsize=16)
for i, mode in enumerate(modes_vmdpy):
    plt.subplot(len(modes_vmdpy), 1, i + 1)
    plt.plot(t[:2000], pad_signal(signal[:2000], mode[:2000]), label=f"VMDPy Mode {i+1}", color='blue')
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.grid(True)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# Plot GPU-Accelerated VMD results (separate modes)
plt.figure(figsize=(25, 18))
plt.suptitle("Decomposed Modes: GPU-Accelerated VMD", fontsize=16)
for i, mode in enumerate(modes_gpu):
    plt.subplot(len(modes_gpu), 1, i + 1)
    plt.plot(t[:2000], pad_signal(signal[:2000], mode[:2000]), label=f"GPU Mode {i+1}", color='green')
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.grid(True)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Sample input: Replace 'modes' with your numpy.ndarray
# modes = np.random.randn(8, 1000)  # Example: 8 IMFs with 1000 samples each

def dominant_frequency_sort(modes, sampling_rate=1.0):
    """
    Sort modes by their dominant frequency.
    
    :param modes: numpy.ndarray of shape (n_modes, n_samples)
    :param sampling_rate: Sampling rate of the signal (default: 1.0)
    :return: Modes sorted by dominant frequency (descending order)
    """
    dominant_frequencies = []
    n_samples = modes.shape[1]
    
    for mode in modes:
        # Compute FFT and frequency spectrum
        freqs = np.fft.fftfreq(n_samples, d=1/sampling_rate)
        fft_values = np.abs(fft(mode))
        
        # Get positive frequencies and their corresponding FFT values
        positive_freqs = freqs[:n_samples // 2]
        positive_fft_values = fft_values[:n_samples // 2]
        
        # Find dominant frequency
        dominant_freq = positive_freqs[np.argmax(positive_fft_values)]
        dominant_frequencies.append(dominant_freq)
    
    # Sort indices by dominant frequencies
    print(dominant_frequencies)
    sorted_indices = np.argsort(dominant_frequencies)[::-1]  # Descending order
    return modes[sorted_indices], sorted_indices

# Example usage
sorted_modes, indices = dominant_frequency_sort(modes_gpu, sampling_rate=1000)
# print("Sorted Modes:", sorted_modes)
print("Indices:", indices)


In [ ]:
# Plot GPU-Accelerated VMD results (separate modes)
plt.figure(figsize=(25, 18))
plt.suptitle("Decomposed Modes: GPU-Accelerated VMD", fontsize=16)
for i, mode in enumerate(sorted_modes[::-1]):
    plt.subplot(len(modes_gpu), 1, i + 1)
    plt.plot(t, pad_signal(signal, mode), label=f"GPU Mode {i+1}", color='green')
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.grid(True)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
type(modes_gpu)

In [ ]:
import librosa
import matplotlib.pyplot as plt
import numpy as np

# Load speech signal
signal, sr = librosa.load(r'C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0034.wav', sr=None)

# Compute STFT
stft_result = librosa.stft(signal, n_fft=2048, hop_length=512)
# Compute magnitude
stft_magnitude = np.abs(stft_result)

plt.figure(figsize=(25, 10))
librosa.display.specshow(stft_magnitude,
                         sr=sr, hop_length=512, y_axis='log', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('STFT Spectrogram')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
librosa.display.specshow(stft_magnitude, sr=sr, hop_length=512, y_axis='log', x_axis='time', ax=ax, cmap='gray')

# Remove the axes
ax.axis('off')

# Remove the borders and save the image
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Adjust to remove padding
# plt.savefig("spectrogram.png", bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
canvas = FigureCanvas(fig)
canvas.draw()

width, height = canvas.get_width_height()9090
image = np.frombuffer(canvas.buffer_rgba(), dtype='uint8').reshape(height, width, 1)

plt.close(fig)

In [ ]:
image.shape

In [ ]:
(a.delaxes()).savefig(r'C:\Users\Arun\parkinson-s-classify\HC_A1_0034.png')

In [ ]:
type(a)

In [ ]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize

# Load speech signal
signal, sr = librosa.load(r'C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0034.wav', sr=None)

# Compute STFT
stft_result = librosa.stft(signal, n_fft=2048, hop_length=512)

# Compute magnitude
stft_magnitude = np.abs(stft_result)

# Convert to decibel scale
spectrogram_db = librosa.amplitude_to_db(stft_magnitude, ref=np.max)

# Resize the spectrogram to 224x224
resized_spectrogram = resize(spectrogram_db, (224, 224), anti_aliasing=True)

# Plot the original spectrogram
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
librosa.display.specshow(spectrogram_db, sr=sr, hop_length=512, y_axis='log', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Original STFT Spectrogram')

# Plot the resized spectrogram
plt.subplot(1, 2, 2)
plt.imshow(resized_spectrogram, origin='lower', aspect='auto', cmap='magma')
plt.colorbar(format='%+2.0f dB')
plt.title('Resized Spectrogram (224x224)')

plt.tight_layout()
plt.show()

# Optional: If you want to save the resized spectrogram as an image
import matplotlib.image as mpimg
mpimg.imsave('resized_spectrogram.png', resized_spectrogram, cmap='magma')

In [ ]:
stft_magnitude.shape

In [ ]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize

# Load speech signal
signal, sr = librosa.load(r'C:\Users\Arun\parkinson-s-classify\Outputs\Reconstructed\HC_A1_0034_reconstructed.wav', sr=None)

# Compute STFT
stft_result = librosa.stft(signal, n_fft=2048, hop_length=512)

# Compute magnitude
stft_magnitude = np.abs(stft_result)

# Convert to decibel scale
spectrogram_db = librosa.amplitude_to_db(stft_magnitude, ref=np.max)

# Resize the spectrogram to 224x224
resized_spectrogram = resize(spectrogram_db, (224, 224), anti_aliasing=True)

# Plot the original spectrogram
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
librosa.display.specshow(spectrogram_db, sr=sr, hop_length=512, y_axis='mel', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Original STFT Spectrogram')

# Plot the resized spectrogram
plt.subplot(1, 2, 2)
plt.imshow(resized_spectrogram, origin='lower', aspect='auto', cmap='viridis')
plt.colorbar(format='%+2.0f dB')
plt.title('Resized Spectrogram (224x224)')

plt.tight_layout()
plt.show()

# Optional: If you want to save the resized spectrogram as an image
# import matplotlib.image as mpimg
# mpimg.imsave('resized_spectrogram.png', resized_spectrogram, cmap='viridis')

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

# Load speech signal
signal, sr = librosa.load(r'C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0034.wav', sr=None)

# Compute STFT
stft_result = librosa.stft(signal, n_fft=2048, hop_length=512)

# Compute magnitude
stft_magnitude = np.abs(stft_result)

# Convert to decibel scale (log scale)
spectrogram_db = librosa.amplitude_to_db(stft_magnitude, ref=np.max)

# Original frequency values (Hz)
freqs = librosa.fft_frequencies(sr=sr, n_fft=2048)

# Target log-scaled frequency bins
log_freqs = np.geomspace(freqs[1], freqs[-1], num=224)

# Interpolate to map original spectrogram to new log scale
interpolator = interp1d(freqs, spectrogram_db, axis=0, bounds_error=False, fill_value=-80)
log_spectrogram = interpolator(log_freqs)

# Plot the original spectrogram
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
librosa.display.specshow(spectrogram_db, sr=sr, hop_length=512, y_axis='log', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Original Log-Scale Spectrogram')

# Plot the resized spectrogram
plt.subplot(1, 2, 2)
plt.imshow(log_spectrogram, origin='lower', aspect='auto', cmap='magma',
           extent=[0, spectrogram_db.shape[1] * (512 / sr), log_freqs[0], log_freqs[-1]])
plt.yscale('log')
plt.colorbar(format='%+2.0f dB')
plt.title('Resized Log-Scale Spectrogram (224x224)')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.tight_layout()
plt.show()

In [ ]:
import librosa
import matplotlib.pyplot as plt
import numpy as np

# Load speech signal
signal, sr = librosa.load(r'C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0034.wav', sr=None)

# Compute STFT
stft_result = librosa.stft(signal, n_fft=2048, hop_length=512)
# Compute magnitude
stft_magnitude = np.abs(stft_result)

plt.figure(figsize=(25, 10))
librosa.display.specshow(librosa.amplitude_to_db(stft_magnitude, ref=np.max),
                         sr=sr, hop_length=512, y_axis='log', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('STFT Spectrogram')
plt.show()

In [ ]:

import librosa
import matplotlib.pyplot as plt
import numpy as np

# Load speech signal
signal, sr = librosa.load(r'C:\Users\Arun\parkinson-s-classify\VMD\filtered_speech.wav', sr=None)

# Compute STFT
stft_result = librosa.stft(signal, n_fft=2048, hop_length=512)
# Compute magnitude
stft_magnitude = np.abs(stft_result)

plt.figure(figsize=(25, 10))
librosa.display.specshow(librosa.amplitude_to_db(stft_magnitude, ref=np.max),
                         sr=sr, hop_length=512, y_axis='log', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('STFT Spectrogram')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import stft

# Step 1: Read the audio file
samplerate, data = wavfile.read(r'C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0034.wav')

# Ensure the audio is mono (convert if necessary)
if data.ndim > 1:
    data = data.mean(axis=1)

# Step 2: Apply STFT
# Define parameters for STFT
nperseg = 1024  # Length of each segment
f, t, Zxx = stft(data, fs=samplerate, nperseg=nperseg)

# Step 3: Filter frequencies up to 1000 Hz
freq_limit = 4000
freq_indices = f <= freq_limit
f_filtered = f[freq_indices]
Zxx_filtered = Zxx[freq_indices, :]

# Step 4: Plot the spectrogram
plt.figure(figsize=(10, 6))
plt.pcolormesh(t, f_filtered, np.abs(Zxx_filtered), shading='gouraud', cmap='inferno')
plt.title('Spectrogram (STFT) - Frequencies up to 1000 Hz')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.colorbar(label='Magnitude')
plt.tight_layout()
plt.show()

In [3]:
import cupy as cp
import numpy as np

def VMD_GPU(f, alpha, tau, K, DC=False, init=0, tol=1e-6):
    """
    GPU-Accelerated Variational Mode Decomposition
    """
    # Ensure input is on GPU and convert to single precision
    f = cp.asarray(f, dtype=cp.float64)
    
    if len(f) % 2:
        f = f[:-1]

    # Period and sampling frequency of input signal
    fs = cp.float32(1. / len(f))
    
    ltemp = len(f) // 2 
    fMirr = cp.concatenate([cp.flip(f[:ltemp]), f, cp.flip(f[-ltemp:])])

    # Time Domain
    T = len(fMirr)
    t = cp.arange(0, T, dtype=cp.float32) / T
    
    # Spectral Domain discretization
    freqs = cp.fft.fftshift(cp.fft.fftfreq(T, d=1.0))

    # Maximum number of iterations
    Niter = 500
    # Individual alpha for each mode
    Alpha = cp.full(K, alpha, dtype=cp.float32)
    
    # Construct and center f_hat
    f_hat = cp.fft.fftshift(cp.fft.fft(fMirr.astype(cp.complex64)))
    f_hat_plus = f_hat.copy()
    f_hat_plus[:T//2] = 0

    # Initialization of omega_k
    omega_plus = cp.zeros((Niter, K), dtype=cp.float32)

    if init == 1:
        omega_plus[0, :] = cp.linspace(0, 0.5, K, dtype=cp.float32)
    elif init == 2:
        omega_plus[0, :] = cp.sort(cp.exp(cp.log(fs) + (cp.log(0.5) - cp.log(fs)) * cp.random.rand(K, dtype=cp.float32)))
    else:
        omega_plus[0, :] = 0
            
    # if DC mode imposed, set its omega to 0
    if DC:
        omega_plus[0, 0] = 0
    
    # Start with empty dual variables
    lambda_hat = cp.zeros((Niter, len(freqs)), dtype=cp.complex64)
    
    # Other initializations
    uDiff = tol + cp.finfo(cp.float32).eps
    n = 0
    sum_uk = 0
    u_hat_plus = cp.zeros((Niter, len(freqs), K), dtype=cp.complex64)

    # Main loop for iterative updates
    while (uDiff > tol and n < Niter - 1):
        # Update first mode accumulator
        k = 0
        sum_uk = u_hat_plus[n, :, K-1] + sum_uk - u_hat_plus[n, :, 0]
        
        # Update spectrum of first mode
        u_hat_plus[n + 1, :, k] = (f_hat_plus - sum_uk - lambda_hat[n, :] / 2) / \
                                  (1 + Alpha[k] * (freqs - omega_plus[n, k]) ** 2)
        
        # Update omega if not held at 0
        if not DC:
            omega_plus[n + 1, k] = cp.dot(
                freqs[T//2:T], cp.abs(u_hat_plus[n + 1, T//2:T, k]) ** 2) / \
                cp.sum(cp.abs(u_hat_plus[n + 1, T//2:T, k]) ** 2)

        # Update any other mode
        for k in range(1, K):
            sum_uk = u_hat_plus[n + 1, :, k - 1] + sum_uk - u_hat_plus[n, :, k]
            u_hat_plus[n + 1, :, k] = (f_hat_plus - sum_uk - lambda_hat[n, :] / 2) / \
                                      (1 + Alpha[k] * (freqs - omega_plus[n, k]) ** 2)
            omega_plus[n + 1, k] = cp.dot(
                freqs[T//2:T], cp.abs(u_hat_plus[n + 1, T//2:T, k]) ** 2) / \
                cp.sum(cp.abs(u_hat_plus[n + 1, T//2:T, k]) ** 2)
            
        # Dual ascent
        lambda_hat[n + 1, :] = lambda_hat[n, :] + tau * (cp.sum(u_hat_plus[n + 1, :, :], axis=1) - f_hat_plus)
        
        # Update loop counter and check for convergence
        n += 1
        uDiff = cp.sum(cp.abs(u_hat_plus[n, :, :] - u_hat_plus[n - 1, :, :]) ** 2).get()
            
    # Postprocessing and cleanup
    Niter = min(Niter, n)
    omega = omega_plus[:Niter, :].astype(cp.float32)
    
    # Reconstruct modes
    u_hat = cp.zeros((T, K), dtype=cp.complex64)
    u_hat[T//2:T, :] = u_hat_plus[Niter - 1, T//2:T, :]
    u_hat[:T//2, :] = cp.conj(u_hat_plus[Niter - 1, T//2:T, :])
    
    u = cp.zeros((K, len(t)), dtype=cp.float32)
    for k in range(K):
        u_k = cp.real(cp.fft.ifft(cp.fft.ifftshift(u_hat[:, k])))
        # Apply proper window function to reduce edge effects
        window = cp.hanning(len(u_k))
        u[k, :] = u_k * window
    
    # 6. Final trimming - ensure proper alignment with original signal
    mid_point = len(u[0]) // 2
    half_len = len(f) // 2
    u = u[:, mid_point-half_len:mid_point+half_len]
    
    return u.get(), u_hat.get(), omega.get()



# Example usage
def test_vmd_gpu():
    # Generate a sample signal
    import numpy as np
    
    # Create a sample signal with multiple modes
    t = np.linspace(0, 1, 1000)
    signal = (np.sin(2 * np.pi * 10 * t) +  # 10 Hz component
              0.5 * np.sin(2 * np.pi * 20 * t) +  # 20 Hz component
              0.25 * np.random.normal(size=t.shape))  # some noise
    
    # Decompose the signal
    modes, mode_spectra, mode_frequencies = VMD_GPU(
        signal, 
        alpha=2000,  # high alpha for tight mode bounds
        tau=0,       # noise-slack
        K=3,         # number of modes to extract
        DC=True,     # keep zero frequency mode
        init=1,      # uniformly distributed initial omegas
        tol=1e-6     # convergence tolerance
    )
    
    return modes, mode_spectra, mode_frequencies

# Uncomment to run the test
# modes, spectra, frequencies = test_vmd_gpu()

# Optional visualization function
def plot_vmd_modes(t, signal, modes):
    """
    Plot the original signal and the decomposed modes.
    
    Parameters:
    t (numpy array): Time values
    signal (numpy array): Original signal
    modes (list of numpy arrays): Decomposed modes
    """
    plt.figure(figsize=(25, 4))
    plt.plot(t, signal, label="Original Signal", color='black')
    plt.plot(t, signal - np.sum(modes, axis=0), label="Residual Signal", color='red')
    
    for i, mode in enumerate(modes):
        plt.plot(t, mode, label=f"VMDPy Mode {i+1}", linewidth=1)
    
    plt.legend()
    plt.title("Decomposed Modes: VMDPy (CPU Implementation)")
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.show()

In [ ]:
# Import the function
import matplotlib.pyplot as plt
import numpy as np

# Generate or load your signal
# t = np.linspace(0, 1, 1000)
# signal = np.sin(2 * np.pi * 10 * t) + 0.5 * np.sin(2 * np.pi * 20 * t) + 0.25 * np.random.normal(size=t.shape)
t = np.linspace(0, 1, 200000)
signal = np.sin(2 * np.pi * 10 * t) + 0.5 * np.sin(2 * np.pi * 50 * t)
# Decompose the signal
modes, mode_spectra, mode_frequencies = VMD(
    signal, 
    alpha=200000,  # adjust as needed
    tau=0,       # noise-slack
    K=2,         # number of modes
    DC=False,     # keep zero frequency mode
    init=2,      # uniformly distributed initial omegas
    tol=1e-30     # convergence tolerance
)

# Optional: Visualize the modes
plot_vmd_modes(t, signal, modes,)

In [ ]:
# Import the function
import matplotlib.pyplot as plt
import numpy as np

# Generate or load your signal
# t = np.linspace(0, 1, 1000)
# signal = np.sin(2 * np.pi * 10 * t) + 0.5 * np.sin(2 * np.pi * 20 * t) + 0.25 * np.random.normal(size=t.shape)
t = np.linspace(0, 1, 200000)
signal = np.sin(2 * np.pi * 10 * t) + 0.5 * np.sin(2 * np.pi * 50 * t)
# Decompose the signal
vmd = GPU_VMD(
        alpha=200000,  # Bandwidth constraint
        tau=0,       # Noise-tolerance
        n_modes=2,   # Number of modes to extract
        dc_component=False,  # Whether to force first mode to be DC
        init_method=1,      # Frequency initialization method
        tolerance=1e-6,     # Convergence tolerance
        device='cuda'       # Computing device
    )
# Decompose signal
result = vmd.decompose(signal)


modes = result.modes

# Optional: Visualize the modes
plot_vmd_modes(t, signal, modes)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the time array
fs = 1000  # Sampling frequency in Hz
t = np.linspace(0, 1, fs, endpoint=False)  # Time vector from 0 to 1 second

# Define the signal components
signal1 = np.sin(2 * np.pi * 10 * t)  # 10 Hz sine wave
signal2 = 0.5 * np.sin(2 * np.pi * 50 * t)  # 50 Hz sine wave with 0.5 amplitude

# Combine the signals
combined_signal = signal1 + signal2

plt.figure(figsize=(25, 4))

# Plot the 10 Hz component
plt.plot(t, signal1, label="10 Hz Component", color="blue", alpha=0.7)

# Plot the 50 Hz component
plt.plot(t, signal2, label="50 Hz Component", color="orange", alpha=0.7)

# Plot the combined signal
plt.plot(t, combined_signal, label="Combined Signal", color="green", linewidth=1.5)

plt.title("Signal Components and Combined Signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [2]:
import librosa
import librosa.display
from main import GPU_VMD
from scipy.io import wavfile


import numpy as np
import matplotlib.pyplot as plt

def plot_vmd_modes_with_spectrum(t, signal, modes, sampling_rate):
    """
    Plot the original signal, decomposed modes in time domain, and their frequency spectra.
    
    Parameters:
    t (numpy array): Time values
    signal (numpy array): Original signal
    modes (list of numpy arrays): Decomposed modes
    sampling_rate (float): Sampling rate of the signal (Hz)
    """
    num_modes = len(modes)
    fig, axes = plt.subplots(num_modes, 2, figsize=(12, 3 * num_modes))

    # Iterate over each mode to plot time-domain and frequency-domain representations
    for i, mode in enumerate(modes):
        # Plot time-domain waveform
        axes[i, 0].plot(t, mode, label=f"IMF{i+1}", color='blue', linewidth=1)
        axes[i, 0].set_title(f"IMF{i+1} - Time Domain")
        axes[i, 0].set_xlabel("Time")
        axes[i, 0].set_ylabel("Amplitude")
        axes[i, 0].grid(True)
        axes[i, 0].legend()

        # Compute frequency spectrum
        freq = np.fft.rfftfreq(len(mode), d=1/sampling_rate)
        spectrum = np.abs(np.fft.rfft(mode))
        
        # Plot frequency spectrum
        axes[i, 1].plot(freq, spectrum, label=f"Frequency Spectrum of IMF{i+1}", color='green', linewidth=1)
        axes[i, 1].set_title(f"IMF{i+1} - Frequency Domain")
        axes[i, 1].set_xlabel("Frequency (Hz)")
        axes[i, 1].set_ylabel("Amplitude")
        axes[i, 1].grid(True)
        axes[i, 1].legend()

    plt.tight_layout()
    plt.show()


# Optional visualization function for spectrograms
def plot_spectrogram(y, sr, title):
    """
    Plot the spectrogram of a given signal using Librosa.
    
    Parameters:
    y (numpy array): Audio signal
    sr (int): Sampling rate
    title (str): Title of the plot
    """
    plt.figure(figsize=(10, 4))
    S = librosa.stft(y)  # Short-Time Fourier Transform
    S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)  # Convert amplitude to dB
    librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='log', cmap='magma')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.show()

# Initialize the VMD
vmd = GPU_VMD(
    alpha=200000,  # Bandwidth constraint
    tau=0,  # Noise-tolerance
    n_modes=8,  # Number of modes to extract
    dc_component=False,  # Whether to force first mode to be DC
    init_method=1,  # Frequency initialization method
    tolerance=1e-6,  # Convergence tolerance
    device='cuda'  # Computing device
)




In [ ]:
# Read the audio file
sr, data = wavfile.read(r'C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0034.wav')

# Normalize the audio data to float32
if data.dtype != np.float32:
    data = data / np.max(np.abs(data), axis=0)  # Normalize to range [-1.0, 1.0]
    data = data.astype(np.float32)

# Decompose the signal using VMD
result = vmd.decompose(data)
modes = result.modes

t = np.linspace(0, len(data) / sr, len(data))

# Optional: Visualize the modes 
plot_vmd_modes_with_spectrum(t, data, modes, sr)

# Plot spectrogram of the original signal
plot_spectrogram(data, sr, "Spectrogram of Original Signal")

# Plot spectrograms of each mode
for i, mode in enumerate(modes):
    plot_spectrogram(mode, sr, f"Spectrogram of Mode {i + 1}")

In [ ]:
import librosa
import librosa.display
from main import GPU_VMD
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt

def plot_vmd_modes_with_spectrum(t, signal, modes, sampling_rate):
    """ 
    Plot the original signal, decomposed modes in time domain, 
    their frequency spectra, and spectrograms.
    
    Parameters:
    t (numpy array): Time values
    signal (numpy array): Original signal
    modes (list of numpy arrays): Decomposed modes
    sampling_rate (float): Sampling rate of the signal (Hz)
    """
    num_modes = len(modes)
    fig, axes = plt.subplots(num_modes + 1, 3, figsize=(36, 4 * (num_modes + 1)))
    
    # Plot original signal
    axes[0, 0].plot(t[:2000], signal[:2000], label="Original Signal", color='red', linewidth=1)
    axes[0, 0].set_title("Original Signal - Time Domain")
    axes[0, 0].set_xlabel("Time (s)")
    axes[0, 0].set_ylabel("Amplitude")
    axes[0, 0].grid(True)
    axes[0, 0].legend()
    
    # Original signal frequency spectrum
    freq = np.fft.rfftfreq(len(signal), d=1/sampling_rate)
    spectrum = np.abs(np.fft.rfft(signal))
    axes[0, 1].plot(freq, spectrum, label="Original Signal Frequency Spectrum", color='red', linewidth=1)
    axes[0, 1].set_title("Original Signal - Frequency Domain")
    axes[0, 1].set_xlabel("Frequency (Hz)")
    axes[0, 1].set_ylabel("Amplitude")
    axes[0, 1].grid(True)
    axes[0, 1].legend()
    
    # Original signal spectrogram
    S = librosa.stft(signal)
    S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
    librosa.display.specshow(S_db, sr=sampling_rate, x_axis='time', y_axis='log', 
                             cmap='magma', ax=axes[0, 2])
    axes[0, 2].set_title("Original Signal Spectrogram")
    
    # Iterate over each mode to plot time-domain, frequency-domain, and spectrogram
    for i, mode in enumerate(modes):
        # Plot time-domain waveform
        axes[i+1, 0].plot(t[:2000], mode[:2000], label=f"IMF{i+1}", color='blue', linewidth=1)
        axes[i+1, 0].set_title(f"IMF{i+1} - Time Domain")
        axes[i+1, 0].set_xlabel("Time (s)")
        axes[i+1, 0].set_ylabel("Amplitude")
        axes[i+1, 0].grid(True)
        axes[i+1, 0].legend()
        
        # Compute frequency spectrum
        freq = np.fft.rfftfreq(len(mode), d=1/sampling_rate)
        spectrum = np.abs(np.fft.rfft(mode))
        axes[i+1, 1].plot(freq, spectrum, label=f"Frequency Spectrum of IMF{i+1}", color='green', linewidth=1)
        axes[i+1, 1].set_title(f"IMF{i+1} - Frequency Domain")
        axes[i+1, 1].set_xlabel("Frequency (Hz)")
        axes[i+1, 1].set_ylabel("Amplitude")
        axes[i+1, 1].grid(True)
        axes[i+1, 1].legend()
        
        # Plot mode spectrogram
        S_mode = librosa.stft(mode)
        S_mode_db = librosa.amplitude_to_db(np.abs(S_mode), ref=np.max)
        librosa.display.specshow(S_mode_db, sr=sampling_rate, x_axis='time', y_axis='log', 
                                 cmap='magma', ax=axes[i+1, 2])
        axes[i+1, 2].set_title(f"IMF{i+1} Spectrogram")
    
    plt.tight_layout()
    plt.show()

# Main execution script
def main():
    # Initialize the VMD
    vmd = GPU_VMD(
        alpha=200000,  # Bandwidth constraint
        tau=0,  # Noise-tolerance
        n_modes=20,  # Number of modes to extract
        dc_component=False,  # Whether to force first mode to be DC
        init_method=1,  # Frequency initialization method
        tolerance=1e-6,  # Convergence tolerance
        device='cuda'  # Computing device
    )

    # Read the audio file
    sr, data = wavfile.read(r'C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0034.wav')
    
    # Normalize the audio data to float32
    if data.dtype != np.float32:
        data = data / np.max(np.abs(data), axis=0)  # Normalize to range [-1.0, 1.0]
        data = data.astype(np.float32)
    
    # Decompose the signal using VMD
    result = vmd.decompose(data)
    modes = result.modes
    
    # Create time array
    t = np.linspace(0, len(data) / sr, len(data))
    
    # Visualize the modes with spectrograms
    plot_vmd_modes_with_spectrum(t, data, modes, sr)

if __name__ == "__main__":
    main()

In [ ]:
import torch
import torch.fft
import cupy as cp
import librosa
import matplotlib.pyplot as plt
from main import GPU_VMD
from scipy.io import wavfile
import numpy as np

def gpu_stft(signal, n_fft=2048, hop_length=512):
    """
    Perform GPU-accelerated Short-Time Fourier Transform using PyTorch
    
    Parameters:
    signal (numpy array): Input signal
    n_fft (int): FFT window size
    hop_length (int): Number of samples between successive frames
    
    Returns:
    torch.Tensor: STFT result
    """
    # Convert to torch tensor and move to GPU
    signal_tensor = torch.from_numpy(signal).float().cuda()
    
    # Create window function on GPU
    window = torch.hann_window(n_fft).cuda()
    
    # Perform STFT
    stft_result = torch.stft(
        signal_tensor, 
        n_fft=n_fft, 
        hop_length=hop_length, 
        window=window, 
        center=True, 
        return_complex=True
    )
    
    return stft_result

def plot_gpu_spectrogram(signal, sr, title, n_fft=2048, hop_length=512):
    """
    Plot spectrogram using GPU-accelerated STFT
    
    Parameters:
    signal (numpy array): Input signal
    sr (int): Sampling rate
    title (str): Plot title
    n_fft (int): FFT window size
    hop_length (int): Number of samples between successive frames
    """
    # Compute GPU STFT
    stft_result = gpu_stft(signal, n_fft, hop_length)
    
    # Convert to magnitude and move back to CPU for plotting
    magnitude = torch.abs(stft_result).cpu().numpy()
    
    # Convert to dB scale
    magnitude_db = librosa.amplitude_to_db(magnitude, ref=np.max)
    
    # Plot
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(
        magnitude_db, 
        sr=sr, 
        x_axis='time', 
        y_axis='log', 
        cmap='magma'
    )
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.tight_layout()
    plt.show()

def plot_vmd_modes_with_gpu_spectrum(t, signal, modes, sampling_rate):
    """ 
    Plot the original signal, decomposed modes in time domain, 
    their frequency spectra, and GPU-accelerated spectrograms.
    """
    num_modes = len(modes)
    fig, axes = plt.subplots(num_modes + 1, 3, figsize=(36, 4 * (num_modes + 1)))
    
    # Plot original signal
    axes[0, 0].plot(t, signal, label="Original Signal", color='red', linewidth=1)
    axes[0, 0].set_title("Original Signal - Time Domain")
    axes[0, 0].set_xlabel("Time (s)")
    axes[0, 0].set_ylabel("Amplitude")
    axes[0, 0].grid(True)
    axes[0, 0].legend()
    
    # Original signal frequency spectrum
    freq = np.fft.rfftfreq(len(signal), d=1/sampling_rate)
    spectrum = np.abs(np.fft.rfft(signal))
    axes[0, 1].plot(freq, spectrum, label="Original Signal Frequency Spectrum", color='red', linewidth=1)
    axes[0, 1].set_title("Original Signal - Frequency Domain")
    axes[0, 1].set_xlabel("Frequency (Hz)")
    axes[0, 1].set_ylabel("Amplitude")
    axes[0, 1].grid(True)
    axes[0, 1].legend()
    
    # Original signal spectrogram using GPU
    stft_result = gpu_stft(signal)
    magnitude = torch.abs(stft_result).cpu().numpy()
    magnitude_db = librosa.amplitude_to_db(magnitude, ref=np.max)
    librosa.display.specshow(
        magnitude_db, 
        sr=sampling_rate, 
        x_axis='time', 
        y_axis='log', 
        cmap='magma', 
        ax=axes[0, 2]
    )
    axes[0, 2].set_title("Original Signal Spectrogram")
    
    # Iterate over each mode to plot time-domain, frequency-domain, and spectrogram
    for i, mode in enumerate(modes):
        # Plot time-domain waveform
        axes[i+1, 0].plot(t, mode, label=f"IMF{i+1}", color='blue', linewidth=1)
        axes[i+1, 0].set_title(f"IMF{i+1} - Time Domain")
        axes[i+1, 0].set_xlabel("Time (s)")
        axes[i+1, 0].set_ylabel("Amplitude")
        axes[i+1, 0].grid(True)
        axes[i+1, 0].legend()
        
        # Compute frequency spectrum
        freq = np.fft.rfftfreq(len(mode), d=1/sampling_rate)
        spectrum = np.abs(np.fft.rfft(mode))
        axes[i+1, 1].plot(freq, spectrum, label=f"Frequency Spectrum of IMF{i+1}", color='green', linewidth=1)
        axes[i+1, 1].set_title(f"IMF{i+1} - Frequency Domain")
        axes[i+1, 1].set_xlabel("Frequency (Hz)")
        axes[i+1, 1].set_ylabel("Amplitude")
        axes[i+1, 1].grid(True)
        axes[i+1, 1].legend()
        
        # Plot mode spectrogram using GPU
        mode_stft = gpu_stft(mode)
        mode_magnitude = torch.abs(mode_stft).cpu().numpy()
        mode_magnitude_db = librosa.amplitude_to_db(mode_magnitude, ref=np.max)
        librosa.display.specshow(
            mode_magnitude_db, 
            sr=sampling_rate, 
            x_axis='time', 
            y_axis='log', 
            cmap='magma', 
            ax=axes[i+1, 2]
        )
        axes[i+1, 2].set_title(f"IMF{i+1} Spectrogram")
    
    plt.tight_layout()
    plt.show()

def main():
    # Initialize the VMD
    vmd = GPU_VMD(
        alpha=200000,  # Bandwidth constraint
        tau=0,  # Noise-tolerance
        n_modes=8,  # Number of modes to extract
        dc_component=False,  # Whether to force first mode to be DC
        init_method=1,  # Frequency initialization method
        tolerance=1e-6,  # Convergence tolerance
        device='cuda'  # Computing device
    )

    # Read the audio file
    sr, data = wavfile.read(r'C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0034.wav')
    
    # Normalize the audio data to float32
    if data.dtype != np.float32:
        data = data / np.max(np.abs(data), axis=0)  # Normalize to range [-1.0, 1.0]
        data = data.astype(np.float32)
    
    # Decompose the signal using VMD
    result = vmd.decompose(data)
    modes = result.modes
    
    # Create time array
    t = np.linspace(0, len(data) / sr, len(data))
    
    # Visualize the modes with GPU-accelerated spectrograms
    plot_vmd_modes_with_gpu_spectrum(t, data, modes, sr)

if __name__ == "__main__":
    main()

In [ ]:
import librosa
import librosa.display
from main import GPU_VMD
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt

def plot_vmd_modes_with_spectrum(t, signal, modes, sampling_rate, include_spectrogram=True, signal_plot_len=None):
    """ 
    Plot the original signal, decomposed modes in time domain and their frequency spectra.
    Optionally include spectrograms.
    
    Parameters:
    t (numpy array): Time values
    signal (numpy array): Original signal
    modes (list of numpy arrays): Decomposed modes
    sampling_rate (float): Sampling rate of the signal (Hz)
    include_spectrogram (bool): Whether to include spectrograms in the visualization
    """
    num_modes = len(modes)
    if signal_plot_len is None:
        signal_plot_len=signal.shape[0]
    
    # Adjust subplot configuration based on spectrogram option
    if include_spectrogram:
        fig, axes = plt.subplots(num_modes + 1, 3, figsize=(36, 3 * (num_modes + 1)))
        plot_columns = 3
    else:
        fig, axes = plt.subplots(num_modes + 1, 2, figsize=(24, 3 * (num_modes + 1)))
        plot_columns = 2
    
    # Plot original signal - Time Domain
    axes[0, 0].plot(t[:signal_plot_len], signal[:signal_plot_len], label="Original Signal", color='red', linewidth=1)
    axes[0, 0].set_title("Original Signal - Time Domain")
    axes[0, 0].set_xlabel("Time (s)")
    axes[0, 0].set_ylabel("Amplitude")
    axes[0, 0].grid(True)
    axes[0, 0].legend()
    
    # Original signal frequency spectrum
    freq = np.fft.rfftfreq(len(signal), d=1/sampling_rate)
    spectrum = np.abs(np.fft.rfft(signal))
    axes[0, 1].plot(freq, spectrum, label="Original Signal Frequency Spectrum", color='red', linewidth=1)
    axes[0, 1].set_title("Original Signal - Frequency Domain")
    axes[0, 1].set_xlabel("Frequency (Hz)")
    axes[0, 1].set_ylabel("Amplitude")
    axes[0, 1].grid(True)
    axes[0, 1].legend()
    
    # Original signal spectrogram (if enabled)
    if include_spectrogram:
        S = librosa.stft(signal)
        S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
        librosa.display.specshow(S_db, sr=sampling_rate, x_axis='time', y_axis='log', 
                                 cmap='magma', ax=axes[0, 2])
        axes[0, 2].set_title("Original Signal Spectrogram")
    
    # Iterate over each mode to plot time-domain and frequency-domain
    for i, mode in enumerate(modes):
        # Plot time-domain waveform
        axes[i+1, 0].plot(t[:signal_plot_len], mode[:signal_plot_len], label=f"IMF{i+1}", color='blue', linewidth=1)
        axes[i+1, 0].set_title(f"IMF{i+1} - Time Domain")
        axes[i+1, 0].set_xlabel("Time (s)")
        axes[i+1, 0].set_ylabel("Amplitude")
        axes[i+1, 0].grid(True)
        axes[i+1, 0].legend()
        
        # Compute frequency spectrum
        freq = np.fft.rfftfreq(len(mode), d=1/sampling_rate)
        spectrum = np.abs(np.fft.rfft(mode))
        axes[i+1, 1].plot(freq, spectrum, label=f"Frequency Spectrum of IMF{i+1}", color='green', linewidth=1)
        axes[i+1, 1].set_title(f"IMF{i+1} - Frequency Domain")
        axes[i+1, 1].set_xlabel("Frequency (Hz)")
        axes[i+1, 1].set_ylabel("Amplitude")
        axes[i+1, 1].grid(True)
        axes[i+1, 1].legend()
        
        # Plot mode spectrogram (if enabled)
        if include_spectrogram:
            S_mode = librosa.stft(mode)
            S_mode_db = librosa.amplitude_to_db(np.abs(S_mode), ref=np.max)
            librosa.display.specshow(S_mode_db, sr=sampling_rate, x_axis='time', y_axis='log', 
                                     cmap='magma', ax=axes[i+1, 2])
            axes[i+1, 2].set_title(f"IMF{i+1} Spectrogram")
    
    plt.tight_layout()
    plt.show()

# Main execution script
def main(include_spectrogram=True):
    # Initialize the VMD
    vmd = GPU_VMD(
        alpha=200000,  # Bandwidth constraint
        tau=0,  # Noise-tolerance
        n_modes=8,  # Number of modes to extract
        dc_component=False,  # Whether to force first mode to be DC
        init_method=1,  # Frequency initialization method
        tolerance=1e-6,  # Convergence tolerance
        device='cuda'  # Computing device
    )

    # Read the audio file
    sr, data = wavfile.read(r'C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0034.wav')
    
    # Normalize the audio data to float32
    if data.dtype != np.float32:
        data = data / np.max(np.abs(data), axis=0)  # Normalize to range [-1.0, 1.0]
        data = data.astype(np.float32)
    
    # Decompose the signal using VMD
    result = vmd.decompose(data, data.shape[0])
    modes = result.modes
    
    # Create time array
    t = np.linspace(0, len(data) / sr, len(data))
    
    # Visualize the modes with optional spectrograms
    plot_vmd_modes_with_spectrum(t, data, modes, sr, include_spectrogram, signal_plot_len=2000)

if __name__ == "__main__":
    # You can now choose whether to include spectrograms
    main(include_spectrogram=True)   # For full visualization
    # main(include_spectrogram=False)  # For a more compact view

In [ ]:
import librosa
import librosa.display
from main import GPU_VMD
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt

def plot_vmd_modes_with_spectrum(t, signal, modes, sampling_rate, include_spectrogram=True, signal_plot_len=None):
    """ 
    Plot the original signal, decomposed modes in time domain and their frequency spectra.
    Optionally include spectrograms.
    
    Parameters:
    t (numpy array): Time values
    signal (numpy array): Original signal
    modes (list of numpy arrays): Decomposed modes
    sampling_rate (float): Sampling rate of the signal (Hz)
    include_spectrogram (bool): Whether to include spectrograms in the visualization
    """
    num_modes = len(modes)
    if signal_plot_len is None:
        signal_plot_len=signal.shape[0]
    
    # Adjust subplot configuration based on spectrogram option
    if include_spectrogram:
        fig, axes = plt.subplots(num_modes + 1, 3, figsize=(36, 3 * (num_modes + 1)))
        plot_columns = 3
    else:
        fig, axes = plt.subplots(num_modes + 1, 2, figsize=(24, 3 * (num_modes + 1)))
        plot_columns = 2
    
    # Plot original signal - Time Domain
    axes[0, 0].plot(t[:signal_plot_len], signal[:signal_plot_len], label="Original Signal", color='red', linewidth=1)
    axes[0, 0].set_title("Original Signal - Time Domain")
    axes[0, 0].set_xlabel("Time (s)")
    axes[0, 0].set_ylabel("Amplitude")
    axes[0, 0].grid(True)
    axes[0, 0].legend()
    
    # Original signal frequency spectrum
    freq = np.fft.rfftfreq(len(signal), d=1/sampling_rate)
    spectrum = np.abs(np.fft.rfft(signal))
    axes[0, 1].plot(freq, spectrum, label="Original Signal Frequency Spectrum", color='red', linewidth=1)
    axes[0, 1].set_title("Original Signal - Frequency Domain")
    axes[0, 1].set_xlabel("Frequency (Hz)")
    axes[0, 1].set_ylabel("Amplitude")
    axes[0, 1].grid(True)
    axes[0, 1].legend()
    
    # Original signal spectrogram (if enabled)
    if include_spectrogram:
        S = librosa.stft(signal)
        S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
        librosa.display.specshow(S_db, sr=sampling_rate, x_axis='time', y_axis='log', 
                                 cmap='magma', ax=axes[0, 2])
        axes[0, 2].set_title("Original Signal Spectrogram")
    
    # Iterate over each mode to plot time-domain and frequency-domain
    for i, mode in enumerate(modes):
        # Plot time-domain waveform
        axes[i+1, 0].plot(t[:signal_plot_len], mode[:signal_plot_len], label=f"IMF{i+1}", color='blue', linewidth=1)
        axes[i+1, 0].set_title(f"IMF{i+1} - Time Domain")
        axes[i+1, 0].set_xlabel("Time (s)")
        axes[i+1, 0].set_ylabel("Amplitude")
        axes[i+1, 0].grid(True)
        axes[i+1, 0].legend()
        
        # Compute frequency spectrum
        freq = np.fft.rfftfreq(len(mode), d=1/sampling_rate)
        spectrum = np.abs(np.fft.rfft(mode))
        axes[i+1, 1].plot(freq, spectrum, label=f"Frequency Spectrum of IMF{i+1}", color='green', linewidth=1)
        axes[i+1, 1].set_title(f"IMF{i+1} - Frequency Domain")
        axes[i+1, 1].set_xlabel("Frequency (Hz)")
        axes[i+1, 1].set_ylabel("Amplitude")
        axes[i+1, 1].grid(True)
        axes[i+1, 1].legend()
        
        # Plot mode spectrogram (if enabled)
        if include_spectrogram:
            S_mode = librosa.stft(mode)
            S_mode_db = librosa.amplitude_to_db(np.abs(S_mode), ref=np.max)
            librosa.display.specshow(S_mode_db, sr=sampling_rate, x_axis='time', y_axis='log', 
                                     cmap='magma', ax=axes[i+1, 2])
            axes[i+1, 2].set_title(f"IMF{i+1} Spectrogram")
    
    plt.tight_layout()
    plt.show()

# Main execution script
def main(include_spectrogram=True):
    # Initialize the VMD
    vmd = GPU_VMD(
        alpha=200000,  # Bandwidth constraint
        tau=0,  # Noise-tolerance
        n_modes=5,  # Number of modes to extract
        dc_component=False,  # Whether to force first mode to be DC
        init_method=1,  # Frequency initialization method
        tolerance=1e-6,  # Convergence tolerance
        device='cuda'  # Computing device
    )

    # Read the audio file
    sr, data = wavfile.read(r'C:\Users\Arun\parkinson-s-classify\VMD\filtered_speech.wav')
    
    # Normalize the audio data to float32
    if data.dtype != np.float32:
        data = data / np.max(np.abs(data), axis=0)  # Normalize to range [-1.0, 1.0]
        data = data.astype(np.float32)
    
    # Decompose the signal using VMD
    result = vmd.decompose(data, data.shape[0])
    modes = result.modes
    
    # Create time array
    t = np.linspace(0, len(data) / sr, len(data))
    
    # Visualize the modes with optional spectrograms
    plot_vmd_modes_with_spectrum(t, data, modes, sr, include_spectrogram, signal_plot_len=None)

if __name__ == "__main__":
    # You can now choose whether to include spectrograms
    main(include_spectrogram=True)   # For full visualization
    # main(include_spectrogram=False)  # For a more compact view

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

def compute_and_plot_spectrograms(file1_path, file2_path):
    """
    Read two WAV files, compute their spectrograms, and plot them side by side.
    
    Parameters:
    file1_path (str): Path to the first WAV file
    file2_path (str): Path to the second WAV file
    """
    # Load the audio files
    y1, sr1 = librosa.load(file1_path)
    y2, sr2 = librosa.load(file2_path)
    
    # Compute spectrograms
    # Using librosa's stft (Short-time Fourier Transform)
    D1 = librosa.stft(y1)
    D2 = librosa.stft(y2)
    
    # Convert to decibel scale
    DB1 = librosa.amplitude_to_db(np.abs(D1), ref=np.max)
    DB2 = librosa.amplitude_to_db(np.abs(D2), ref=np.max)
    
    # Create a figure with two subplots
    plt.figure(figsize=(12, 6))
    
    # Plot first spectrogram
    plt.subplot(1, 2, 1)
    librosa.display.specshow(DB1, sr=sr1, x_axis='time', y_axis='hz')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f'Spectrogram of {file1_path.split("/")[-1]}')
    
    # Plot second spectrogram
    plt.subplot(1, 2, 2)
    librosa.display.specshow(DB2, sr=sr2, x_axis='time', y_axis='hz')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f'Spectrogram of {file2_path.split("/")[-1]}')
    
    # Adjust layout and show plot
    plt.tight_layout()
    plt.show()

# Example usage
if __name__ == "__main__":
    # Replace these paths with the actual paths to your WAV files
    file1 = r"C:\Users\Arun\parkinson-s-classify\VMD\filtered_speech.wav"
    file2 = r"C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0034.wav"
    
    compute_and_plot_spectrograms(file1, file2)

In [ ]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import wiener
import matplotlib.pyplot as plt

# Step 1: Read the noisy speech signal
sample_rate, noisy_signal = wavfile.read(r'C:\Users\Arun\parkinson-s-classify\neurovoz_v3\data\audios\HC_A1_0034.wav')

# Check the signal properties
print(f"Sample Rate: {sample_rate} Hz")
print(f"Signal Length: {len(noisy_signal)} samples")

# Normalize the signal if it's in int16 format
if noisy_signal.dtype == np.int16:
    noisy_signal = noisy_signal / np.max(np.abs(noisy_signal))

# Step 2: Apply Wiener filtering
filtered_signal = wiener(noisy_signal)

# Step 3: Save the filtered output
wavfile.write('filtered_speech.wav', sample_rate, np.int16(filtered_signal * 32767))

# Plot the original and filtered signals for comparison
plt.figure(figsize=(12, 6))

plt.subplot(2, 1, 1)
plt.title('Noisy Speech Signal')
plt.plot(noisy_signal, color='red')
plt.xlabel('Sample')
plt.ylabel('Amplitude')

plt.subplot(2, 1, 2)
plt.title('Filtered Speech Signal (Wiener)')
plt.plot(filtered_signal, color='green')
plt.xlabel('Sample')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()


In [6]:
from main import GPU_VMD, save_modes_and_reconstruct
from scipy.signal import wiener
import librosa
import numpy as np

vmd = GPU_VMD(
        alpha=200000,
        tau=0,
        n_modes=20,
        dc_component=False,
        init_method=1,
        tolerance=1e-10,
        device='cuda'
    )
# # Load the audio file
audio_path = r'E:\Amrita\Subjects\Sem 5\BMSP paper work\Dataset\neurovoz_v3\data\audios\HC_A1_0049.wav'
data, sr = librosa.load(audio_path)
data = wiener(data)

# file_path = r'C:\Users\Arun\Downloads\OneDrive_1_12-13-2024\Folder 1\arctic_a0004.wav'
# data, sr = librosa.load(file_path, mono=False)

# data = data[0]
result = vmd.decompose(data, data.shape[0])
modes = result.modes

# Pad each mode to the same length of the original signal
modes_padded = []
for mode in modes:
    mode_padded = np.pad(mode, (0, len(data) - len(mode)), 'constant')
    modes_padded.append(mode_padded)

# Save the decomposed modes and reconstructed signal
save_modes_and_reconstruct(modes_padded, sr, data, 'summa1')


0.12819673374699317